# Metrocar Funnel Exploration 

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:

# import the  data

app_downloads = pd.read_csv('metrocar_data/app_downloads.csv')
reviews = pd.read_csv('metrocar_data/reviews.csv')
ride_requests = pd.read_csv('metrocar_data/ride_requests.csv')
signups = pd.read_csv('metrocar_data/signups.csv')
transactions = pd.read_csv('metrocar_data/transactions.csv')


/var/folders/1m/hvnyjbcx2rxfb9d__rb_m7h00000gp/T/ipykernel_17546/1537861751.py:5: DtypeWarning: Columns (4,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ride_requests = pd.read_csv('metrocar_data/ride_requests.csv')


In [3]:
# create the SQLite connection

cnn = sqlite3.connect('cars.db')

In [4]:
# Save the data to SQLite

app_downloads.to_sql('app_downloads', cnn, if_exists='replace')
reviews.to_sql('reviews', cnn, if_exists='replace')
ride_requests.to_sql('ride_requests', cnn, if_exists='replace')
signups.to_sql('signups', cnn, if_exists='replace')
transactions.to_sql('transactions', cnn, if_exists='replace')

223652

In [5]:
# Load the ipython-sql extension
%load_ext sql

# Connect to the SQLite database
%sql sqlite:///cars.db

'Connected: @cars.db'

In [6]:
%%sql

SELECT COUNT(*)
FROM app_downloads;

 * sqlite:///cars.db
Done.


COUNT(*)
23608


### Dataset structure

Description of each table and its columns.

#### app_downloads: contains information about app downloads
* app_download_key: unique id of an app download
* platform: ios, android or web
* download_ts: download timestamp


#### signups: contains information about new user signups

* user_id: primary id for a user
* session_id: id of app download
* signup_ts: signup timestamp
* age_range: the age range the user belongs to



#### ride_requests: contains information about rides


* ride_id: primary id for a ride
* user_id: foreign key to user (requester)
* driver_id: foreign key to driver
* request_ts: ride request timestamp
* accept_ts: driver accept timestamp
* pickup_location: pickup coordinates
* destination_location: destination coordinates
* pickup_ts: pickup timestamp
* dropoff_ts: dropoff timestamp
* cancel_ts: ride cancel timestamp (accept, pickup and dropoff timestamps may be null)


#### transactions: contains information about financial transactions based on completed rides:


* ride_id: foreign key to ride
* purchase_amount_usd: purchase amount in USD
* charge_status: approved, cancelled
* transaction_ts: transaction timestamp


#### reviews: contains information about driver reviews once rides are completed


* review_id: primary id of review
* ride_id: foreign key to ride
* driver_id: foreign key to driver
* user_id: foreign key to user (requester)
* rating: rating from 0 to 5
* free_response: text response given by user/requester

How many times was the app downloaded?


In [34]:
%%sql
WITH avg_ratings AS (SELECT user_id, ROUND(AVG(rating),2) avg_rating
FROM reviews
GROUP BY user_id)

SELECT AVG(avg_rating)
FROM avg_ratings

 * sqlite:///cars.db
Done.


AVG(avg_rating)
3.0628403863845386


In [8]:
%%sql

SELECT COUNT(*) Number_of_downloads
FROM app_downloads;

 * sqlite:///cars.db
Done.


Number_of_downloads
23608


How many users signed up on the app?

In [9]:
%%sql

SELECT COUNT (DISTINCT user_id) AS total_user_signups
FROM signups

 * sqlite:///cars.db
Done.


total_user_signups
17623


How many rides were requested through the app?


In [10]:
%%sql

SELECT COUNT(request_ts) AS total_requests
FROM ride_requests


 * sqlite:///cars.db
Done.


total_requests
385477


How many unique users requested a ride through the Metrocar app?

In [11]:
%%sql

SELECT COUNT(DISTINCT user_id) AS total_unique_requests
FROM ride_requests
WHERE request_ts > 0 

 * sqlite:///cars.db
Done.


total_unique_requests
12406


How many unique users completed a ride through the Metrocar app?

In [12]:
%%sql

SELECT COUNT(DISTINCT user_id) AS total_unique_rides
FROM ride_requests
WHERE dropoff_ts > 0 

 * sqlite:///cars.db
Done.


total_unique_rides
6233


How many rides were requested and completed through the app?


In [13]:
%%sql

SELECT COUNT(accept_ts) AS total_accepts, COUNT(dropoff_ts) AS total_completed_requests
FROM ride_requests

 * sqlite:///cars.db
Done.


total_accepts,total_completed_requests
248379,223652


How many rides were requested and how many unique users requested a ride?


In [14]:
%%sql

SELECT COUNT(ride_id) total_requests, COUNT(DISTINCT user_id) total_users
FROM ride_requests

 * sqlite:///cars.db
Done.


total_requests,total_users
385477,12406


What is the average time of a ride from pick up to drop off?


In [15]:
ride_requests.head()

,ride_id,user_id,driver_id,request_ts,accept_ts,pickup_location,dropoff_location,pickup_ts,dropoff_ts,cancel_ts
0,3080556,108995,116266.0,2021-07-26 09:01:00,2021-07-26 09:19:00,40.72216744 -73.96212375,40.72262277 -73.87869592,NaN,NaN,2021-07-26 09:25:00
1,3081967,110902,106286.0,2021-08-23 16:42:00,2021-08-23 16:59:00,40.74316496 -73.90019974,40.79460742 -73.81512246,NaN,NaN,2021-08-23 17:09:00
2,3088174,114998,116029.0,2021-11-13 17:06:00,2021-11-13 17:23:00,40.76639545 -73.877075,40.75548354 -73.9505886,NaN,NaN,2021-11-13 17:33:00
3,3180652,112421,109517.0,2021-10-16 15:28:00,2021-10-16 15:47:00,40.76499488 -73.8481648,40.86129334 -73.89895067,NaN,NaN,2021-10-16 15:52:00
4,3191244,104964,106628.0,2021-05-07 19:31:00,2021-05-07 19:51:00,40.73396733 -73.79521329,40.88348849 -74.03331643,NaN,NaN,2021-05-07 19:58:00


In [16]:
%%sql

SELECT ROUND(AVG((strftime('%s', dropoff_ts) - strftime('%s', pickup_ts)) / 60),2) AS average_duration_mins
FROM ride_requests

 * sqlite:///cars.db
Done.


average_duration_mins
52.61


How many rides were accepted by a driver?


In [17]:
%%sql

SELECT COUNT(accept_ts) AS total_accepted_requests
FROM ride_requests

 * sqlite:///cars.db
Done.


total_accepted_requests
248379


How many rides did we successfully collect payments and how much was collected?


In [18]:
%%sql

SELECT COUNT(transaction_ts) AS total_transations, ROUND(SUM(purchase_amount_usd), 2) AS total_usd_collected
FROM transactions
WHERE charge_status LIKE 'approved'

 * sqlite:///cars.db
Done.


total_transations,total_usd_collected
212628,4251667.61


In [19]:
%%sql

SELECT COUNT(DISTINCT user_id)
FROM transactions t
LEFT JOIN ride_requests r
ON r.ride_id = t.ride_id
WHERE charge_status LIKE 'Approved'

 * sqlite:///cars.db
Done.


COUNT(DISTINCT user_id)
6233


How many ride requests happened on each platform?


In [20]:
%%sql

SELECT platform, COUNT(request_ts) total_requests
FROM app_downloads a
LEFT JOIN signups s
ON a.app_download_key = s.session_id
LEFT JOIN ride_requests r
ON r.user_id = s.user_id
GROUP BY 1

 * sqlite:///cars.db
Done.


platform,total_requests
android,112317
ios,234693
web,38467


What is the drop-off from users signing up to users requesting a ride?

Reference: https://popsql.com/sql-templates/marketing/running-a-funnel-analysis#calculating-drop-off-at-each-step

In [21]:
%%sql

WITH steps AS (
    SELECT 'Sign Up' as step, COUNT(DISTINCT user_id) as count FROM signups
    UNION
    SELECT 'Ride Request' as step, COUNT(request_ts) as count FROM ride_requests
)


SELECT step, 
        count, 
        lag(count, 1) OVER() as previous_count,
        ROUND((1.0 - count * 1.0 / lag(count) OVER (ORDER BY step)), 2) AS drop_off

from steps;

 * sqlite:///cars.db
Done.


step,count,previous_count,drop_off
Ride Request,385477,None,None
Sign Up,17623,385477,0.95


In [22]:
app_downloads['download_ts'].min(), app_downloads['download_ts'].max()

('2021-01-01 00:05:59', '2021-12-31 23:52:27')

In [23]:
%%sql

WITH funnel AS (SELECT
  'App Download' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(download_ts) AS user_count,
  NULL AS ride_count
FROM app_downloads d
LEFT JOIN signups s ON s.session_id = d.app_download_key
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Signup' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT s.user_id) AS user_count,
  NULL AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Ride Request' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Driver Acceptance' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
WHERE r.accept_ts IS NOT NULL
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Completed Ride' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
INNER JOIN transactions t ON t.ride_id = r.ride_id
WHERE r.cancel_ts IS NULL AND t.charge_status = 'Approved'
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Completed Payment' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
INNER JOIN transactions t ON t.ride_id = r.ride_id
WHERE r.cancel_ts IS NULL AND t.charge_status = 'Approved'
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Completed Review' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
INNER JOIN transactions t ON t.ride_id = r.ride_id
INNER JOIN reviews rev ON rev.ride_id = t.ride_id
WHERE r.cancel_ts IS NULL AND t.charge_status = 'Approved' AND rev.rating IS NOT NULL
GROUP BY download_date, d.platform, s.age_range
ORDER BY step, platform, age_range, download_date)

SELECT SUM(user_count)
FROM funnel
WHERE step LIKE 'App Download';

 * sqlite:///cars.db
Done.


SUM(user_count)
23608


In [24]:
# aggregate the data to a user level and export as .csv for analysis in Tableau 

df = pd.read_sql_query("""
SELECT
  'App Download' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT download_ts) AS user_count,
  NULL AS ride_count
FROM app_downloads d
LEFT JOIN signups s ON s.session_id = d.app_download_key
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Signup' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT s.user_id) AS user_count,
  NULL AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Ride Request' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Driver Acceptance' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
WHERE r.accept_ts IS NOT NULL
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Completed Ride' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
INNER JOIN transactions t ON t.ride_id = r.ride_id
WHERE r.cancel_ts IS NULL AND t.charge_status = 'Approved'
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Completed Payment' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
INNER JOIN transactions t ON t.ride_id = r.ride_id
WHERE r.cancel_ts IS NULL AND t.charge_status = 'Approved'
GROUP BY download_date, d.platform, s.age_range

UNION

SELECT
  'Completed Review' AS step,
  DATE(d.download_ts) AS download_date,
  d.platform,
  s.age_range,
  COUNT(DISTINCT r.user_id) AS user_count,
  COUNT(DISTINCT r.ride_id) AS ride_count
FROM app_downloads d
INNER JOIN signups s ON s.session_id = d.app_download_key
INNER JOIN ride_requests r ON r.user_id = s.user_id
INNER JOIN transactions t ON t.ride_id = r.ride_id
INNER JOIN reviews rev ON rev.ride_id = t.ride_id
WHERE r.cancel_ts IS NULL AND t.charge_status = 'Approved' AND rev.rating IS NOT NULL
GROUP BY download_date, d.platform, s.age_range
ORDER BY step, platform, age_range, download_date
;""", cnn)



In [25]:
df.to_csv("metrocars_agg_dataset.csv", index=False)

df.shape

(26901, 6)

In [26]:
%%sql

WITH downloads AS (
  SELECT
    app_download_key,
    MIN(download_ts) AS min_time
  FROM app_downloads
  GROUP BY app_download_key
),
sign_ups AS (
  SELECT DISTINCT
    s.session_id,
    s.user_id
  FROM downloads d
  INNER JOIN signups s ON s.session_id = d.app_download_key
),
requests AS (
  SELECT DISTINCT
    rr.user_id
  FROM sign_ups s
  INNER JOIN ride_requests rr ON s.user_id = rr.user_id
),
accepted_requests AS (
  SELECT DISTINCT
    rr.user_id
  FROM requests r
  INNER JOIN ride_requests rr ON r.user_id = rr.user_id
  WHERE rr.accept_ts IS NOT NULL
),
completed_rides AS (
  SELECT DISTINCT
    rr.user_id,
    rr.ride_id
  FROM accepted_requests a
  INNER JOIN ride_requests rr ON a.user_id = rr.user_id
  WHERE rr.cancel_ts IS NULL
  GROUP BY 1
),
completed_payments AS (
  SELECT 
    t.ride_id, r.user_id,
    COUNT(DISTINCT r.user_id) AS completed_transactions_count
  FROM completed_rides r
  INNER JOIN transactions t ON r.ride_id = t.ride_id
  WHERE t.charge_status LIKE 'Approved'
  GROUP BY t.ride_id
),
completed_reviews AS (
  SELECT 
    p.ride_id
  FROM completed_payments p
  INNER JOIN reviews r ON p.ride_id = r.ride_id
  WHERE r.rating IS NOT NULL
),
steps AS (
  SELECT 'App Download' AS step, COUNT(*) AS count
  FROM downloads
  UNION
  SELECT 'Signup' AS step, COUNT(*) AS count
  FROM sign_ups
  UNION
  SELECT 'Request Ride' AS step, COUNT(*) AS count
  FROM requests
  UNION
  SELECT 'Driver Acceptance' AS step, COUNT(*) AS count
  FROM accepted_requests
  UNION
  SELECT 'Completed Ride' AS step, COUNT(*) AS count
  FROM completed_rides
  UNION
  SELECT 'Completed Payment' AS step, COUNT(*) AS count
  FROM completed_payments
  UNION
  SELECT 'Completed Review' AS step, COUNT(*) AS count
  FROM completed_reviews
  ORDER BY count DESC
),
previous_counts AS (
  SELECT
    step,
    count,
    LAG(count) OVER () AS previous_count

  FROM steps
)
SELECT
  step,
  count,
  previous_count,
  ROUND((CAST(count AS REAL) / previous_count)*100, 2) AS conversion,
  ROUND((1.0 - CAST(count AS REAL) / previous_count)*100, 2) AS drop_off,
  ROUND((CAST(count AS REAL) / (SELECT Max(count) FROM steps)) * 100, 2) AS percent_of_top


FROM previous_counts;


 * sqlite:///cars.db
Done.


step,count,previous_count,conversion,drop_off,percent_of_top
App Download,23608,None,None,None,100.0
Signup,17623,23608,74.65,25.35,74.65
Request Ride,12406,17623,70.4,29.6,52.55
Driver Acceptance,12278,12406,98.97,1.03,52.01
Completed Ride,6233,12278,50.77,49.23,26.4
Completed Payment,5962,6233,95.65,4.35,25.25
Completed Review,4163,5962,69.83,30.17,17.63
